In [7]:
import pathlib
import pandas as pd
import numpy as np
import re
import time
#import fasttext
#fasttext.FastText.eprint = lambda x: None
#from huggingface_hub import hf_hub_download
from tqdm import tqdm

In [3]:
def get_lang(
    df: pd.DataFrame,
    col_calculate_on: str
) -> str:
    """Detects the language of a text column in a DataFrame using langdetect.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the text column
    col_calculate_on : str
        Column name to calculate language on

    Returns
    -------
    df : pd.DataFrame
        DataFrame with the language column added to it as "lang"
    """

    def det(x: str) -> str:
        """
        Detects the language of a given text

        Parameters
        ----------
        x : str
            Text whose language is to be detected

        Returns
        -------
        lang : str
            Language of the text
        """

        try:
            lang = model.predict(x)[0][0].replace('__label__','')
        except Exception as e:
            print(e)
            lang = 'Other'
        return lang

    print(f"-- Detecting language...")
    start_time = time.time()

    df['lang'] = df[col_calculate_on].apply(det)

    print(f'-- -- Language detect finished in {(time.time() - start_time)}')

    return df

def normalize_string(s):
    # Convert to lower case
    #s = s.lower()
    # Remove punctuation and special characters
    #s = re.sub(r'[^\w\s]', '', s)
    # Remove extra spaces
    s = re.sub(r'\s+', ' ', s).strip()
    return s

Estos ficheros se generan con el notebook "Procesado_Textos_BSC".

No obstante, algo no cuadra: 
- Montse dijo que había **155.980** documentos en **catalán**
- De los documentos que nos pasan, sólo aparecen **3.942** (esto lo obtenemos si leemos ```/export/data_ml4ds/NextProcurement/PLACE/pdf2txt_ca_translated``` y hacemos ``combined_df_ca[(combined_df_ca['translated_content'] != "") & (pd.notna(combined_df_ca['translated_content']))]`` o ``combined_df_ca[combined_df_ca.alternative_lang == "ca"]``. 
- Si nos quedamos con los que son ténicos, sólo quedan **839** documentos traducidos del catalán. ¿No son pocos?


**Repuesta de BSC de cómo detectan el lenguaje:** Para la identificación de idiomas de nextProcurement usamos el modelo de Fasttext: https://fasttext.cc/docs/en/language-identification.html

En nuestra implementación, ya que el modelo solo puede clasificar pequeñas porciones de texto, no clasificamos el documento entero directamente, si no que vamos frase por frase.

Entonces, para cada frase en el documento, obtenemos el idioma con mayor probabilidad, calculada por el modelo. Con las frases clasificadas por idiomas, calculamos el porcentaje de carácteres en cada idioma. Finalmente, clasificamos el documento con la lengua que tenga más del 60% de carácteres. Cuando no hay ninguna lengua que sobrepase este 60%, los documentos se clasifican como "mx". En "mx" es normal encontrar documentos con lenguas minoritarias que el modelo no sabe clasificar bien (gallego, catalán) y que además contienen código o puntuación excesiva.

**Nosotros de momento vamos a aplicar fastext en el texto sin más, a la espera de que BSC nos pase los idiomas que ellos han detectado.**

Han hecho algo raro con los nuevos parquets que nos han pasado traducidos. Al hacer el extract de los que no han sido traducidos, muchos aparecen vacíos. Seguramente se puede resolver el fallo revisando el código de extracción, pero de momento vamos a trabajar haciendo un merge de los que ya teníamos bien procesado del dump anterior (``BSC_procesados``) con los nuevos procesados tras la traducción (``BSC_procesados_ca``).

**Había un error con lo que nos habían pasado traducidos en catalán. Nos solucionan el error y al mismo tiempo nos proporcionan la traducción de los pliegos en gallego y euskera.**

Correo de Jens: "Efectivamente había un problema de nuestro lado. Un script de preparación de datos que utilizamos antes de pasar el traductor ha cambiado el orden de los documentos y como resultado no todas las traducciones se habían incorporado correctamente en los ficheros Parquet.

Hemos cambiado la forma de combinar las traducciones con los Parquets existentes y el resultado parece correcto ahora. Hemos integrado también las traducciones de los textos en gallego y vasco. Los ficheros están subidos a nextp:PLACE/pdf2txt_all_translated y parecen incluir todas las traducciones:

documentos totales: **4114855**

documentos traducidos:
- ca: 166251
- gl: 25572
- eu: 2291

Cuidado que los campo de "alternative_lang" y "translated_content" para los documentos no traducidos puede tener el valor None / null, o puede ser un string vacío. Para evitar cualquier problema recomendamos mirar expresamente que "alternative_lang" sea "ca", "gl", o "eu" y usar "translated_content" solo en estos casos."

In [4]:
166251 + 25572 + 2291

194114

In [8]:
path_all = pathlib.Path("/export/data_ml4ds/NextProcurement/PLACE/BSC_procesados") # path to all
path_ca_tr = pathlib.Path("/export/data_ml4ds/NextProcurement/PLACE/Septiembre2024_procesados_all") # path to detected as Catalan and translated #BSC_procesados_ca #pdf2txt_ca_translated

In [4]:
#model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
#model = fasttext.load_model(model_path) 

In [9]:
# Read ALL
paths = []
paths = [path for path in path_all.iterdir()]
all_dfs = []
for path_ in tqdm(paths[:1]):
    df = pd.read_parquet(path_)
    all_dfs.append(df)
combined_df = pd.concat(all_dfs)

100%|██████████| 1/1 [01:26<00:00, 86.40s/it]


In [26]:
combined_df["len"] = combined_df["extracted"].apply(lambda x : len(x))
combined_df[combined_df.len < 100]

,procurement_id,doc_name,extracted,extracted_tags,texto_heading,embeddings_heading,len
458,ntp01085779,ntp01085779_Pliego_Prescripciones_tecnicas_URI,dE\n UN\n \n ...\nI* * *I\nas\n,"[document, section, heading, body, p, p]",[dE\n UN\n \n ...],"[[-0.1700753, 0.13922103, -0.13927573, -0.1512...",25
3591,ntp01299513,ntp01299513_Pliego_Prescripciones_tecnicas_URI,FICHA DE CARACTERISTICAS\n,"[document, section, heading]",[FICHA DE CARACTERISTICAS],"[[0.14469801, 0.059316613, -0.014979365, 0.112...",25
4123,ntp11391384,ntp11391384_Pliego_Prescripciones_tecnicas_URI,FOR-01_PA-03 ed1 13/12/2021\nPág. 1 de 2\n,"[document, section, heading, section, heading]","[FOR-01_PA-03 ed1 13/12/2021, Pág. 1 de 2]","[[-0.47894797, 0.049413368, -0.05345886, -0.04...",40
5611,ntp11502746,ntp11502746_Pliego_Prescripciones_tecnicas_URI,FOR-01_PA-03 ed1 13/12/2021\nPág. 1 de 2\n,"[document, section, heading, section, heading]","[FOR-01_PA-03 ed1 13/12/2021, Pág. 1 de 2]","[[-0.47894797, 0.049413368, -0.05345886, -0.04...",40
13093,ntp00859372,ntp00859372_Pliego_Prescripciones_tecnicas_URI,https://obras.dip-caceres.es/proyectos/2020-12...,"[document, section, heading]",[https://obras.dip-caceres.es/proyectos/2020-1...,"[[-0.1865231, 0.11410366, -0.095274046, -0.196...",54
...,...,...,...,...,...,...,...
242227,ntp11273132,ntp11273132_Pliego_Prescripciones_tecnicas_URI...,7\n ILUSTRE\n AYUNTAMIENTO\n DE LA ALDEA\n \n ...,"[document, section, heading, section, heading]",[7\n ILUSTRE\n AYUNTAMIENTO\n DE LA ALDEA\n \n...,"[[0.15344131, 0.22321746, 0.10389321, 0.032998...",74
245416,ntp01022967,ntp01022967_Pliego_Prescripciones_tecnicas_URI,https://obras.dip-caceres.es/proyectos/2021-1-...,"[document, section, heading]",[https://obras.dip-caceres.es/proyectos/2021-1...,"[[-0.21282248, 0.06790182, -0.085010596, -0.14...",51
246579,ntp00918302,ntp00918302_Pliego_Prescripciones_tecnicas_URI,https://obras.dip-caceres.es/proyectos/2021-1-...,"[document, section, heading]",[https://obras.dip-caceres.es/proyectos/2021-1...,"[[-0.16487059, 0.06240326, -0.09061762, -0.138...",51
249784,ntp01188640,ntp01188640_Pliego_Prescripciones_tecnicas_URI,HAY QUE PRESENTAR FICHAS TÉCNICAS Y FOTOGRAFIA...,"[document, section, heading]",[HAY QUE PRESENTAR FICHAS TÉCNICAS Y FOTOGRAFI...,"[[-0.039869603, 0.1789974, 0.08720622, 0.00361...",73


In [36]:
df_df_to_merge = pd.read_parquet("/export/data_ml4ds/NextProcurement/PLACE/pliegos_objectives/all_extracted_31_oct_es_translated_with_place_id_created24feb_merge_with_place.parquet")

In [37]:
df_df_to_merge.columns

Index(['doc_name', 'translated', 'extracted', 'clean_extracted', 'lang',
       'objective', 'in_text_score', 'processed_0', 'processed_1000',
       'processed_2000', 'processed_3000', 'processed_4000', 'procurement_id',
       'alternative_lang', 'lang_objective', 'place_id',
       'ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode',
       'ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentityCode',
       'ContractFolderStatus.LocatedContractingParty.ParentLocatedParty.PartyName.Name',
       'ContractFolderStatus.ProcurementProject.BudgetAmount.EstimatedOverallContractAmount',
       'ContractFolderStatus.TenderResult.SMEAwardedIndicator',
       'ContractFolderStatus.ContractFolderID', 'title',
       'ContractFolderStatus.ValidNoticeInfo.AdditionalPublicationStatus.AdditionalPublicationDocumentReference.IssueDate',
       'origin', 'similarity'],
      dtype='object')

In [38]:
df_df_to_merge.head(2)

,doc_name,translated,extracted,clean_extracted,lang,objective,in_text_score,processed_0,processed_1000,processed_2000,...,ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode,ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentityCode,ContractFolderStatus.LocatedContractingParty.ParentLocatedParty.PartyName.Name,ContractFolderStatus.ProcurementProject.BudgetAmount.EstimatedOverallContractAmount,ContractFolderStatus.TenderResult.SMEAwardedIndicator,ContractFolderStatus.ContractFolderID,title,ContractFolderStatus.ValidNoticeInfo.AdditionalPublicationStatus.AdditionalPublicationDocumentReference.IssueDate,origin,similarity
0,ntp00114200_Pliego_Prescripciones_tecnicas_URI,no,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n \...,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS S - CON...,spa_Latn,El objeto del presente Pliego es la definición...,1.0,True,True,True,...,[33111000.0],ES120,<NA>,28000.0,None,NS221/18,Suministro de Detector de Equipo de RX Portáti...,"[['2018-02-06', '2018-02-06'], ['2018-03-05', ...",out,0.641474
1,ntp00729859_Pliego_Prescripciones_tecnicas_URI,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,spa_Latn,"El objeto del contrato, correspondiente a la p...",1.0,True,True,True,...,[50330000.0],ES,Mutua Midat Cyclops,498030.0,[nan],N202000910,Prestación de Servicios de Explotación de los ...,"[['2020-12-21', '2020-12-16', '2020-12-28'], [...",ins,0.247979


In [ ]:
df_df_gencat = pd.read_parquet("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Backend-Dockers/data/source/place_all_embeddings_metadata.parquet")

In [33]:
df_df_gencat.columns

Index(['id', 'place_id', 'origin', 'expedient', 'objective', 'cpv', 'nuts',
       'org_loc', 'budget', 'date', 'place_date', 'link', 'raw_text', 'lemmas',
       'embeddings', 'title_origin', 'title'],
      dtype='object')

In [35]:
df_df_gencat.head(2)

,id,place_id,origin,expedient,objective,cpv,nuts,org_loc,budget,date,place_date,link,raw_text,lemmas,embeddings,title_origin,title
0,0,https://contrataciondelestado.es/sindicacion/P...,out,2019_213,L'objecte d'aquest encàrrec per part del CONSO...,None,ES512,Entitats de l'administració local,NaN,2025-03-17,[nan],https://contractaciopublica.cat/ca/detall-publ...,L'objecte d'aquest encàrrec per part del CONSO...,consorci_vies verdes_girona tragsa conservació...,-0.23268852 -0.2679665 -0.072283626 -0.1170956...,metadata,None
1,1,https://contrataciondelestado.es/sindicacion/l...,ins,DCG-307/2019,Proyecto De Adecuación De Oficinas De Los Serv...,[45000000.0],ES300,Aena. Dirección de Contratación,NaN,2025-03-17,[nan],https://contrataciondelestado.es/wps/poc?uri=d...,Proyecto De Adecuación De Oficinas De Los Serv...,proyecto adecuación oficinas servicios_central...,-0.02814787 -0.13409117 -0.06640905 -0.0788602...,metadata,None


In [55]:
df_anal = combined_df.merge(df_df_to_merge[["procurement_id", "place_id"]], how="left", on="procurement_id")
df_anal2 = df_anal.merge(df_df_gencat[["place_id", "link", "origin", "objective"]], how="left", on="place_id")
df_anal2_no_nan = df_anal2[df_anal2.place_id != np.nan]
df_anal2_no_nan = df_anal2.dropna(subset=['place_id', 'link'])
keep = df_anal2_no_nan[df_anal2_no_nan.len < 100][['procurement_id', 'doc_name', 'place_id', 'link', 'extracted', 'extracted_tags',
       'texto_heading', 'len', 'origin', 'objective']]

In [58]:
keep.to_csv("bsc_failed.csv")

In [59]:
keep

,procurement_id,doc_name,place_id,link,extracted,extracted_tags,texto_heading,len,origin,objective
645,ntp11502746,ntp11502746_Pliego_Prescripciones_tecnicas_URI,https://contrataciondelestado.es/sindicacion/d...,https://contrataciondelestado.es/wps/poc?uri=d...,FOR-01_PA-03 ed1 13/12/2021\nPág. 1 de 2\n,"[document, section, heading, section, heading]","[FOR-01_PA-03 ed1 13/12/2021, Pág. 1 de 2]",40,min,Alquiler de material de iluminación para la Es...
2161,ntp00029499,ntp00029499_Pliego_Prescripciones_tecnicas_URI...,https://contrataciondelestado.es/sindicacion/P...,http://www.madrid.org/cs/Satellite?op2=PCON&id...,w Junta de\n Castilla y León\nRESOLUCIÓN DE AP...,"[document, section, body, p, section, heading]",[RESOLUCIÓN DE APROBACIÓN DELGASTO -\n \n ...\...,78,out,"Adquisición de medicamentos: Cinacalcet, Romip..."
3570,ntp00831544,ntp00831544_Pliego_Prescripciones_tecnicas_URI...,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,ñmma\n ocedemento Aberto Simplificado Procedem...,"[document, section, heading]",[ñmma\n ocedemento Aberto Simplificado Procede...,94,ins,Ejecución de las obras del proyecto de sustitu...
8698,ntp00322607,ntp00322607_Pliego_Prescripciones_tecnicas_URI...,https://contrataciondelestado.es/sindicacion/P...,http://www.madrid.org/cs/Satellite?op2=PCON&id...,MINISTERIO SUBSECRETARÍA .\n DE HAFII:I\IF\I\ ...,"[document, section, heading]",[MINISTERIO SUBSECRETARÍA .\n DE HAFII:I\IF\I\...,85,out,Adquisición de la obra de arte “Restauració de...
13866,ntp01000821,ntp01000821_Pliego_Prescripciones_tecnicas_URI...,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,PARADORES\nHoteles 8 Restaurantes 1928\n,"[document, section, heading, body, p]",[PARADORES],38,ins,obra de construcción de nuevo depósito de 500 ...
22206,ntp00617111,ntp00617111_Pliego_Prescripciones_tecnicas_URI...,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,DOCUMENTO\n .Acta: 79662 25753 - Acta mesa Con...,"[document, section, heading]",[DOCUMENTO\n .Acta: 79662 25753 - Acta mesa Co...,65,ins,Contrato de arrendamiento de trece licencias d...
23226,ntp00881599,ntp00881599_Pliego_Prescripciones_tecnicas_URI...,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,\nCORRECION DE ERRORES\nN? de E;?y- !:-—+- lan...,"[document, section, body, p, section, heading,...","[CORRECION DE ERRORES, LO]",69,ins,Suministro de mobiliario para el Campus Duques...
23643,ntp11631525,ntp11631525_Pliego_Prescripciones_tecnicas_URI...,https://contrataciondelestado.es/sindicacion/d...,https://contrataciondelestado.es/wps/poc?uri=d...,Expediente: BOLANOS2021/15312\nDECRETO\n,"[document, section, body, p, section, heading]",[DECRETO],38,min,Suministro de cremas solares
23866,ntp00233912,ntp00233912_Pliego_Prescripciones_tecnicas_URI...,https://contrataciondelestado.es/sindicacion/P...,https://www.contratacion.euskadi.eus/w32-kpepe...,_7 —\n MANC ANR\n,"[document, section, heading]",[_7 —\n MANC ANR],15,out,Suministro de una máquina ultracentrífuga con ...
28137,ntp00531325,ntp00531325_Pliego_Prescripciones_tecnicas_URI...,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,AYUNTAMIENTO DE OTERO DE HERREROS\nExpediente ...,"[document, section, heading, body, p]",[AYUNTAMIENTO DE OTERO DE HERREROS],64,ins,Edificio para gimnasio municipal


In [ ]:
# Read CA
paths = []
paths = [path for path in path_ca_tr.iterdir()]
all_dfs = []
for path_ in tqdm(paths):
    df = pd.read_parquet(path_)
    all_dfs.append(df)
combined_df_ca = pd.concat(all_dfs)

In [10]:
combined_df_ca

,procurement_id,doc_name,extracted,alternative_lang,translated
207,ntp00289491,ntp00289491_Pliego_Prescripciones_tecnicas_URI,arquitectos MERCADÉ. C/Nou DE SANTA TECLA 9 2o...,ca,yes
331,ntp00372307,ntp00372307_Pliego_Prescripciones_tecnicas_URI,Departamento de Seguridad Dirección de Adminis...,eu,yes
523,ntp00405990,ntp00405990_Pliego_Prescripciones_tecnicas_URI,Ayuntamiento de Barcelona Movilidad e Infraest...,ca,yes
744,ntp00339949,ntp00339949_Pliego_Prescripciones_tecnicas_URI,¿;%'é/z Instituto de Formación Continua-IL3 E ...,ca,yes
781,ntp00297383,ntp00297383_Pliego_Prescripciones_tecnicas_URI,Ayuntamiento de La Pobla de Mafumet PLIEGO TÉC...,ca,yes
...,...,...,...,...,...
251990,ntp00438282,ntp00438282_Pliego_Prescripciones_tecnicas_URI,PLIEGO DE PRESCRIPCIONES TECNICAS QUE HAN DE R...,,no
252027,ntp00569941,ntp00569941_Pliego_Prescripciones_tecnicas_URI,ET%;: MINISTERIO _- Centro de investigaciones\...,,no
252033,ntp01097618,ntp01097618_Pliego_Prescripciones_tecnicas_URI,Ayuntamiento de Puerto de la Cruz\nPLIEGO DE P...,,no
252042,ntp01346184,ntp01346184_Pliego_Prescripciones_tecnicas_URI,Ref.: RTam\nAYUNTAMIENTO DE CHESTE\nPLIEGO DE ...,,no


In [ ]:
combined_df_ca[(combined_df_ca.alternative_lang == "ca") | (combined_df_ca.alternative_lang == "eu") | (combined_df_ca.alternative_lang == "ga")]

In [12]:
combined_df_ca[(combined_df_ca.alternative_lang == "ga")]

,procurement_id,doc_name,extracted,alternative_lang,translated


In [13]:
combined_df_ca.columns

Index(['procurement_id', 'doc_name', 'extracted', 'alternative_lang',
       'translated'],
      dtype='object')

In [14]:
combined_df_ca[combined_df_ca.translated == "yes"]

,procurement_id,doc_name,extracted,alternative_lang,translated
207,ntp00289491,ntp00289491_Pliego_Prescripciones_tecnicas_URI,arquitectos MERCADÉ. C/Nou DE SANTA TECLA 9 2o...,ca,yes
331,ntp00372307,ntp00372307_Pliego_Prescripciones_tecnicas_URI,Departamento de Seguridad Dirección de Adminis...,eu,yes
523,ntp00405990,ntp00405990_Pliego_Prescripciones_tecnicas_URI,Ayuntamiento de Barcelona Movilidad e Infraest...,ca,yes
744,ntp00339949,ntp00339949_Pliego_Prescripciones_tecnicas_URI,¿;%'é/z Instituto de Formación Continua-IL3 E ...,ca,yes
781,ntp00297383,ntp00297383_Pliego_Prescripciones_tecnicas_URI,Ayuntamiento de La Pobla de Mafumet PLIEGO TÉC...,ca,yes
...,...,...,...,...,...
251188,ntp00436474,ntp00436474_Pliego_Prescripciones_tecnicas_URI,il K G VICEPRESIDENCIA O Y CONSELLERIA E INNOV...,ca,yes
251234,ntp00221835,ntp00221835_Pliego_Prescripciones_tecnicas_URI,AYUNTAMIENTO DE ESPARREGUERA AYUNTAMIENTO DE E...,ca,yes
251267,ntp00965595,ntp00965595_Pliego_Prescripciones_tecnicas_URI,PLIEGO DE PRESCRIPCIONES TECNICAS PARTICULARES...,ca,yes
251735,ntp00565817,ntp00565817_Pliego_Prescripciones_tecnicas_URI,v AYUNTAMIENTO DE CIUTADELLA DE MENORCA Pliego...,ca,yes


In [15]:
merged_df = pd.merge(combined_df, combined_df_ca, on='doc_name', how='inner')

merged_df['extracted'] = merged_df.apply(
    lambda row: row['extracted_y'] if row['alternative_lang'] == 'ca' else row['extracted_x'], axis=1)

merged_df = merged_df.drop(columns=['extracted_x', 'extracted_y', 'procurement_id_y'])
merged_df

,procurement_id_x,doc_name,extracted_tags,texto_heading,embeddings_heading,alternative_lang,translated,extracted
0,ntp00114200,ntp00114200_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n ...,"[[-0.1855995, 0.10908591, 0.1287027, -0.085297...",,no,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n \...
1,ntp00729859,ntp00729859_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE R...,"[[-0.06488852, 0.03319587, 0.05720649, -0.2051...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...
2,ntp01335443,ntp01335443_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[0.11980498, -0.0047292123, 0.07927151, -0.05...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...
3,ntp00008052,ntp00008052_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[íHJJ Agencia de I'Habitatge\n FUU de Cataluny...,"[[0.050423447, 0.10861048, 0.12632288, 0.06832...",ca,yes,íHJJ Agencia de la Vivienda FUU de Cataluña PL...
4,ntp00196761,ntp00196761_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[Agencia de Vivienda Social\n CONSEJERÍA DE VI...,"[[0.2535557, 0.06023139, 0.13293944, 0.0011294...",,no,Ref: 50/088288.9/21\nAgencia de Vivienda Socia...
...,...,...,...,...,...,...,...,...
429833,ntp00582515,ntp00582515_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[-0.101150624, 0.29571363, -0.01729959, 0.204...",,no,sura\nPLIEGO DE PRESCRIPCIONES TÉCNICAS PARA L...
429834,ntp01351598,ntp01351598_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...","[COLEGIO DE INGENIEROS DE CAMINOS,\n CANALES Y...","[[-0.041178588, -0.10482038, 0.20265633, 0.086...",,no,2020/02293/01 14/07/2020\n \n ...\nCOLEGIO DE ...
429835,ntp00566851,ntp00566851_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...","[ANEX, Características técnicas de la aplicaci...","[[-0.02871781, 0.109616816, -0.03430078, 0.037...",,no,ANEX\nPliego de prescripciones técnicas que ha...
429836,ntp00882149,ntp00882149_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[UNIVERSITAT\n POLITÉCNICA\n DE VALENCIA\n \n ...,"[[0.04755034, -0.071341686, 0.07659534, 0.1800...",,no,a\n A\n S\nUNIVERSITAT\n POLITÉCNICA\n DE VALE...


In [ ]:
merged_df["clean_extracted"] = merged_df["extracted"].apply(normalize_string)
merged_df = get_lang(merged_df, col_calculate_on = "clean_extracted")

In [14]:
merged_df.to_parquet("/export/data_ml4ds/NextProcurement/PLACE/pliegos_extracted/all_extracted_29jul.parquet")

In [16]:
merged_df

,procurement_id_x,doc_name,extracted_tags,texto_heading,embeddings_heading,alternative_lang,translated,extracted,clean_extracted,lang
0,ntp00114200,ntp00114200_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n ...,"[[-0.1855995, 0.10908591, 0.1287027, -0.085297...",,no,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n \...,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS S - CON...,spa_Latn
1,ntp00729859,ntp00729859_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE R...,"[[-0.06488852, 0.03319587, 0.05720649, -0.2051...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,spa_Latn
2,ntp01335443,ntp01335443_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[0.11980498, -0.0047292123, 0.07927151, -0.05...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,spa_Latn
3,ntp00008052,ntp00008052_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[íHJJ Agencia de I'Habitatge\n FUU de Cataluny...,"[[0.050423447, 0.10861048, 0.12632288, 0.06832...",,no,íHJJ Agencia de I'Habitatge\n FUU de Catalunya...,íHJJ Agencia de I'Habitatge FUU de Catalunya P...,cat_Latn
4,ntp00196761,ntp00196761_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[Agencia de Vivienda Social\n CONSEJERÍA DE VI...,"[[0.2535557, 0.06023139, 0.13293944, 0.0011294...",,no,Ref: 50/088288.9/21\nAgencia de Vivienda Socia...,Ref: 50/088288.9/21 Agencia de Vivienda Social...,spa_Latn
...,...,...,...,...,...,...,...,...,...,...
455432,ntp00582515,ntp00582515_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[-0.101150624, 0.29571363, -0.01729959, 0.204...",,no,sura\nPLIEGO DE PRESCRIPCIONES TÉCNICAS PARA L...,sura PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA...,spa_Latn
455433,ntp01351598,ntp01351598_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...","[COLEGIO DE INGENIEROS DE CAMINOS,\n CANALES Y...","[[-0.041178588, -0.10482038, 0.20265633, 0.086...",,no,2020/02293/01 14/07/2020\n \n ...\nCOLEGIO DE ...,2020/02293/01 14/07/2020 ... COLEGIO DE INGENI...,spa_Latn
455434,ntp00566851,ntp00566851_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...","[ANEX, Características técnicas de la aplicaci...","[[-0.02871781, 0.109616816, -0.03430078, 0.037...",,no,ANEX\nPliego de prescripciones técnicas que ha...,ANEX Pliego de prescripciones técnicas que ha ...,spa_Latn
455435,ntp00882149,ntp00882149_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[UNIVERSITAT\n POLITÉCNICA\n DE VALENCIA\n \n ...,"[[0.04755034, -0.071341686, 0.07659534, 0.1800...",,no,a\n A\n S\nUNIVERSITAT\n POLITÉCNICA\n DE VALE...,a A S UNIVERSITAT POLITÉCNICA DE VALENCIA ... ...,spa_Latn


In [15]:
sample_df = merged_df.sample(n=1000, random_state=42)
sample_df.to_parquet("/export/data_ml4ds/NextProcurement/PLACE/pliegos_extracted/all_extracted_29jul_sample1000.parquet")

In [17]:
merged_df_es = merged_df[merged_df.lang == "spa_Latn"]

In [18]:
merged_df_es

,procurement_id_x,doc_name,extracted_tags,texto_heading,embeddings_heading,alternative_lang,translated,extracted,clean_extracted,lang
0,ntp00114200,ntp00114200_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n ...,"[[-0.1855995, 0.10908591, 0.1287027, -0.085297...",,no,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n \...,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS S - CON...,spa_Latn
1,ntp00729859,ntp00729859_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE R...,"[[-0.06488852, 0.03319587, 0.05720649, -0.2051...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,spa_Latn
2,ntp01335443,ntp01335443_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[0.11980498, -0.0047292123, 0.07927151, -0.05...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,spa_Latn
4,ntp00196761,ntp00196761_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[Agencia de Vivienda Social\n CONSEJERÍA DE VI...,"[[0.2535557, 0.06023139, 0.13293944, 0.0011294...",,no,Ref: 50/088288.9/21\nAgencia de Vivienda Socia...,Ref: 50/088288.9/21 Agencia de Vivienda Social...,spa_Latn
5,ntp01108524,ntp01108524_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE R...,"[[-0.13603789, 0.22781396, -0.091998294, -0.10...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,spa_Latn
...,...,...,...,...,...,...,...,...,...,...
455432,ntp00582515,ntp00582515_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[-0.101150624, 0.29571363, -0.01729959, 0.204...",,no,sura\nPLIEGO DE PRESCRIPCIONES TÉCNICAS PARA L...,sura PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA...,spa_Latn
455433,ntp01351598,ntp01351598_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...","[COLEGIO DE INGENIEROS DE CAMINOS,\n CANALES Y...","[[-0.041178588, -0.10482038, 0.20265633, 0.086...",,no,2020/02293/01 14/07/2020\n \n ...\nCOLEGIO DE ...,2020/02293/01 14/07/2020 ... COLEGIO DE INGENI...,spa_Latn
455434,ntp00566851,ntp00566851_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...","[ANEX, Características técnicas de la aplicaci...","[[-0.02871781, 0.109616816, -0.03430078, 0.037...",,no,ANEX\nPliego de prescripciones técnicas que ha...,ANEX Pliego de prescripciones técnicas que ha ...,spa_Latn
455435,ntp00882149,ntp00882149_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[UNIVERSITAT\n POLITÉCNICA\n DE VALENCIA\n \n ...,"[[0.04755034, -0.071341686, 0.07659534, 0.1800...",,no,a\n A\n S\nUNIVERSITAT\n POLITÉCNICA\n DE VALE...,a A S UNIVERSITAT POLITÉCNICA DE VALENCIA ... ...,spa_Latn


In [19]:
merged_df_es.to_parquet("/export/data_ml4ds/NextProcurement/PLACE/pliegos_extracted/all_extracted_12aug_es.parquet")

In [20]:
merged_df_es

,procurement_id_x,doc_name,extracted_tags,texto_heading,embeddings_heading,alternative_lang,translated,extracted,clean_extracted,lang
0,ntp00114200,ntp00114200_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n ...,"[[-0.1855995, 0.10908591, 0.1287027, -0.085297...",,no,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n \...,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS S - CON...,spa_Latn
1,ntp00729859,ntp00729859_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE R...,"[[-0.06488852, 0.03319587, 0.05720649, -0.2051...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,spa_Latn
2,ntp01335443,ntp01335443_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[0.11980498, -0.0047292123, 0.07927151, -0.05...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,spa_Latn
4,ntp00196761,ntp00196761_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[Agencia de Vivienda Social\n CONSEJERÍA DE VI...,"[[0.2535557, 0.06023139, 0.13293944, 0.0011294...",,no,Ref: 50/088288.9/21\nAgencia de Vivienda Socia...,Ref: 50/088288.9/21 Agencia de Vivienda Social...,spa_Latn
5,ntp01108524,ntp01108524_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE R...,"[[-0.13603789, 0.22781396, -0.091998294, -0.10...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,spa_Latn
...,...,...,...,...,...,...,...,...,...,...
455432,ntp00582515,ntp00582515_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[-0.101150624, 0.29571363, -0.01729959, 0.204...",,no,sura\nPLIEGO DE PRESCRIPCIONES TÉCNICAS PARA L...,sura PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA...,spa_Latn
455433,ntp01351598,ntp01351598_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...","[COLEGIO DE INGENIEROS DE CAMINOS,\n CANALES Y...","[[-0.041178588, -0.10482038, 0.20265633, 0.086...",,no,2020/02293/01 14/07/2020\n \n ...\nCOLEGIO DE ...,2020/02293/01 14/07/2020 ... COLEGIO DE INGENI...,spa_Latn
455434,ntp00566851,ntp00566851_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...","[ANEX, Características técnicas de la aplicaci...","[[-0.02871781, 0.109616816, -0.03430078, 0.037...",,no,ANEX\nPliego de prescripciones técnicas que ha...,ANEX Pliego de prescripciones técnicas que ha ...,spa_Latn
455435,ntp00882149,ntp00882149_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[UNIVERSITAT\n POLITÉCNICA\n DE VALENCIA\n \n ...,"[[0.04755034, -0.071341686, 0.07659534, 0.1800...",,no,a\n A\n S\nUNIVERSITAT\n POLITÉCNICA\n DE VALE...,a A S UNIVERSITAT POLITÉCNICA DE VALENCIA ... ...,spa_Latn


In [21]:
merged_df_es.columns

Index(['procurement_id_x', 'doc_name', 'extracted_tags', 'texto_heading',
       'embeddings_heading', 'alternative_lang', 'translated', 'extracted',
       'clean_extracted', 'lang'],
      dtype='object')

In [23]:
merged_df_es[["procurement_id_x","doc_name","translated", "extracted","clean_extracted", "lang"]].to_parquet("/export/data_ml4ds/NextProcurement/PLACE/pliegos_extracted/all_extracted_12aug_es.parquet")

## Objectives extracted

In [24]:
import pandas as pd
df = pd.read_parquet("/export/data_ml4ds/NextProcurement/PLACE/pliegos_objectives/all_extracted_12aug_es.parquet")

In [25]:
df

,procurement_id_x,doc_name,translated,extracted,clean_extracted,lang,objective,in_text_score
0,ntp00114200,ntp00114200_Pliego_Prescripciones_tecnicas_URI,no,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n \...,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS S - CON...,spa_Latn,None,None
1,ntp00729859,ntp00729859_Pliego_Prescripciones_tecnicas_URI,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,spa_Latn,None,None
2,ntp01335443,ntp01335443_Pliego_Prescripciones_tecnicas_URI,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,spa_Latn,None,None
4,ntp00196761,ntp00196761_Pliego_Prescripciones_tecnicas_URI,no,Ref: 50/088288.9/21\nAgencia de Vivienda Socia...,Ref: 50/088288.9/21 Agencia de Vivienda Social...,spa_Latn,None,None
5,ntp01108524,ntp01108524_Pliego_Prescripciones_tecnicas_URI,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,spa_Latn,None,None
...,...,...,...,...,...,...,...,...
455432,ntp00582515,ntp00582515_Pliego_Prescripciones_tecnicas_URI,no,sura\nPLIEGO DE PRESCRIPCIONES TÉCNICAS PARA L...,sura PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA...,spa_Latn,None,None
455433,ntp01351598,ntp01351598_Pliego_Prescripciones_tecnicas_URI,no,2020/02293/01 14/07/2020\n \n ...\nCOLEGIO DE ...,2020/02293/01 14/07/2020 ... COLEGIO DE INGENI...,spa_Latn,None,None
455434,ntp00566851,ntp00566851_Pliego_Prescripciones_tecnicas_URI,no,ANEX\nPliego de prescripciones técnicas que ha...,ANEX Pliego de prescripciones técnicas que ha ...,spa_Latn,None,None
455435,ntp00882149,ntp00882149_Pliego_Prescripciones_tecnicas_URI,no,a\n A\n S\nUNIVERSITAT\n POLITÉCNICA\n DE VALE...,a A S UNIVERSITAT POLITÉCNICA DE VALENCIA ... ...,spa_Latn,None,None


In [33]:
df.columns

Index(['procurement_id_x', 'doc_name', 'translated', 'extracted',
       'clean_extracted', 'lang', 'objective', 'in_text_score'],
      dtype='object')